https://www.kaggle.com/hugoncosta/price-of-flats-in-moscow

price - цена квартиры в $1000

totsp - общая площадь квартиры, кв.м.

livesp - жилая площадь квартиры, кв.м.

kitsp - площадь кухни, кв.м.

dist - расстояние от центра в км.

metrdist - расстояние до метро в минутах

walk - 1 – пешком от метро, 0 – на транспорте

brick - 1 – кирпичный, монолит ж/б, 0 – другой

floor - 1 – этаж кроме первого и последнего, 0 – иначе.

code - число от 1 до 8, при помощи которого мы группируем наблюдения по подвыборкам: 1. Наблюдения сгруппированы на севере, вокруг Калужско-Рижской линии метрополитена 2. Север, вокруг Серпуховско-Тимирязевской линии метрополитена 3. Северо-запад, вокруг Замоскворецкой линии метрополитена 4. Северо-запад, вокруг Таганско-Краснопресненской линии метрополитена 5. Юго-восток, вокруг Люблинской линии метрополитена 6. Юго-восток, вокруг Таганско-Краснопресненской линии метрополитена 7. Восток, вокруг Калиниской линии метрополитена 8. Восток, вокруг Арбатско-Покровской линии метрополитена

In [107]:
import pandas as pd
import numpy as np
import dill
from sklearn.metrics import roc_auc_score,roc_curve,scorer
from urllib import request, parse
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score as r2
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import roc_auc_score,roc_curve,scorer
from sklearn.metrics import f1_score
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import precision_score,recall_score

from sklearn.impute import SimpleImputer

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

In [108]:
import sys
!{sys.executable} -m pip install -U pandas

In [109]:
df = pd.read_csv('flats_moscow.csv', index_col=0)

In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2040 entries, 1 to 2040
Data columns (total 10 columns):
price       2040 non-null int64
totsp       2040 non-null int64
livesp      2040 non-null int64
kitsp       2040 non-null float64
dist        2040 non-null float64
metrdist    2040 non-null int64
walk        2040 non-null int64
brick       2040 non-null int64
floor       2040 non-null int64
code        2040 non-null int64
dtypes: float64(2), int64(8)
memory usage: 175.3 KB


In [111]:
df.head()

,price,totsp,livesp,kitsp,dist,metrdist,walk,brick,floor,code
1,81,58,40,6.0,12.5,7,1,1,1,3
2,75,44,28,6.0,13.5,7,1,0,1,6
3,128,70,42,6.0,14.5,3,1,1,1,3
4,95,61,37,6.0,13.5,7,1,0,1,1
5,330,104,60,11.0,10.5,7,0,1,1,3


In [112]:
df.describe()

,price,totsp,livesp,kitsp,dist,metrdist,walk,brick,floor,code
count,2040.000000,2040.000000,2040.000000,2040.000000,2040.000000,2040.000000,2040.000000,2040.000000,2040.000000,2040.000000
mean,127.496569,73.084314,46.337255,8.898529,11.015686,8.117157,0.685784,0.323039,0.790686,4.322059
std,51.878220,15.123450,7.894348,2.787073,3.375539,3.815574,0.464317,0.467752,0.406918,2.183289
min,50.000000,44.000000,28.000000,5.000000,3.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,95.000000,62.000000,42.000000,7.000000,9.000000,5.000000,0.000000,0.000000,1.000000,3.000000
50%,115.000000,73.500000,45.000000,9.000000,12.000000,7.000000,1.000000,0.000000,1.000000,4.000000
75%,142.000000,79.000000,50.000000,10.000000,13.500000,10.000000,1.000000,1.000000,1.000000,6.000000
max,730.000000,192.000000,102.000000,25.000000,17.000000,20.000000,1.000000,1.000000,1.000000,8.000000


In [113]:
df.isnull().sum()

price       0
totsp       0
livesp      0
kitsp       0
dist        0
metrdist    0
walk        0
brick       0
floor       0
code        0
dtype: int64

In [114]:
X_train, X_test, y_train, y_test = train_test_split(df, 
                                                    df['price'], test_size=0.33, random_state=23)
#save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)
#save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

/Users/sonzza/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """
/Users/sonzza/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


In [115]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [122]:
features = ["totsp", "livesp", "kitsp", "dist", 'metrdist', "walk", "brick", "floor", "code"]
target = 'price'

In [123]:
final_transformers = list()

for feature in features:
    num_transformer = Pipeline([
                ('selector', NumberSelector(feature)),
                ('scaler', MinMaxScaler())
            ])
    final_transformers.append((feature, num_transformer))
    
    
feats = FeatureUnion(final_transformers)

feature_processing = Pipeline([('feats', feats)])

In [124]:

LOWER_ALPHA = 0.1
UPPER_ALPHA = 0.9

regressor_mid = Pipeline([
    ('features',feats),
    ('classifier_mid', GradientBoostingRegressor(loss="ls"))
])
    
regressor_low = Pipeline([
    ('features',feats),
    ('classifier_low', GradientBoostingRegressor(loss="quantile",                   
                                        alpha=LOWER_ALPHA))
])

regressor_up = Pipeline([
    ('features',feats),
    ('classifier_up', GradientBoostingRegressor(loss="quantile",
                                        alpha=UPPER_ALPHA))
])

In [125]:
regressor_mid.fit(X_train, y_train)
regressor_low.fit(X_train, y_train)
regressor_up.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('totsp',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='totsp')),
                                                                 ('scaler',
                                                                  MinMaxScaler())])),
                                                ('livesp',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='livesp')),
                                                                 ('scaler',
                                                                  MinMaxScaler())])),
                                                ('kitsp',
                                                 Pipeline(steps=[('selector',
                                             

In [126]:
predictions_mid = regressor_mid.predict(X_test)
predictions_mid
mean_absolute_error(y_pred=predictions_mid, y_true=y_test), r2_score(y_pred=predictions_mid, y_true=y_test)

(15.498145964003182, 0.712962265069773)

In [121]:
predictions_low = regressor_low.predict(X_test)
predictions_low
mean_absolute_error(y_pred=predictions_low, y_true=y_test), r2_score(y_pred=predictions_low, y_true=y_test)

(25.01360974284839, 0.33200069403765775)

In [104]:
predictions_up = regressor_up.predict(X_test)
predictions_up
mean_absolute_error(y_pred=predictions_up, y_true=y_test), r2_score(y_pred=predictions_up, y_true=y_test)

(25.633946681746902, 0.5181447661038048)

In [105]:
with open("regressor_mid.dill", "wb") as f:
    dill.dump(regressor_mid, f)

In [127]:
import urllib.request
import json      

def get_prediction(totsp, livesp, dist, metrdist, walk, brick, floor, code):
    body = {'общая площадь квартиры, кв.м.': totsp, 
            'жилая площадь квартиры, кв.м.': livesp,
            'площадь кухни, кв.м.': kitsp,
            'расстояние от центра в км.': dist,
            'расстояние до метро в минутах': metrdist,
            'Пешком или на транспорте? 1 – пешком, 0 – на транспорте': walk,
            'Дом: 1 - кирпичный, монолит ж/б, 0 – другой': brick,
            '1 – этаж кроме первого и последнего, 0 – иначе': floor,
            'Линия метро:\
            1.  Калужско-Рижская \
            2. Серпуховско-Тимирязевская \
            3. Замоскворецкая \
            4. Таганско-Краснопресненская  \
            5. Люблинская \
            6. Таганско-Краснопресненская \
            7. Калиниская \
            8. Арбатско-Покровская линии': code
           
           } 

    myurl = "http://0.0.0.0:8180/predict"
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    #print (jsondataasbytes)
    response = urllib.request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['predictions']